# Análise exploratória de dados e visualização - Trabalho de A2
#### Pedro Santos Tokar

## Parte A

A base de dados escolhida para esse projeto é uma base sobre música, mais especificamente sobre as vendas da indústria musical dos Estados Unidos desde 1973. Ela está disponível no site [Kaggle](https://www.kaggle.com/datasets/andrewmvd/music-sales).

Antes de me firmar nessa base de dados, eu considerei usar uma base de dados a respeito do [perfil dos moradores de rua](https://obpoprua.direito.ufmg.br/repositorio_dados.html#tabelas_capitais) nas capitais brasileiras, mais especificamente com uma ideia de comparar o perfil dos moradores de rua de São Paulo/SP e do Rio de Janeiro/RJ. Essa ideia acabou abandonada por mim porque as proporções dos perfis (Faixas etárias, gênero, etc.) dos sem teto das duas capitais eram praticamente as mesmas.

A base de dados das vendas da indústria musical não apresenta nenhuma divisão baseada na música sendo vendida, mas sim no meio em que essa música foi vendida. Logo, no lugar de perguntas relacionadas a música como "Quais gêneros vendem mais?" ou "O que caracteriza as músicas de cada época?", as perguntas que podem ser feitas são do tipo "Qual mídia vendeu mais em determinado período?" ou "Quando a mídia x deixou de existir?". Essas e outras perguntas podem dar dicas de como se moveu e funcinou o mercado musical (no caráter das mídias) em certos períodos.

Além de perguntas específicas, é possível generalizar mais as dúvidas que podem surgir sobre esse assunto: **"Como é a história da indústria musical americana? Como cada formato se saiu no mercado?"**. Essas são as perguntas que serão exploradas na peça gráfica feita para esse trabalho: como foi o comportamento desse mercado ao longo do tempo.

Como a base de dados é organizada por períodos (anos) e em formatos, responder esse tipo de pergunta é possível, e é interessante que o gráfico feito tenha um foco na passagem de tempo, e não represente apenas um período, já que isso acaba com o propósito de contar a história da indústria.

## Parte B

Antes de tudo, é preciso carregar a base de dados e as bibliotecas pandas e altair, que serão usadas para fazer a análise exploratória:

In [1]:
import pandas as pd
import altair as alt
import numpy as np
dados = pd.read_csv("musicdata.csv")
dados

,format,metric,year,number_of_records,value_actual
0,CD,Units,1973,1,NaN
1,CD,Units,1974,1,NaN
2,CD,Units,1975,1,NaN
3,CD,Units,1976,1,NaN
4,CD,Units,1977,1,NaN
...,...,...,...,...,...
3003,Vinyl Single,Value (Adjusted),2015,1,6.205390
3004,Vinyl Single,Value (Adjusted),2016,1,5.198931
3005,Vinyl Single,Value (Adjusted),2017,1,6.339678
3006,Vinyl Single,Value (Adjusted),2018,1,5.386197


Como é possível ver, a tabela tem 5 colunas: `format`, `metric`, `year`, `number_of_records`, `value_actual`. É de se supor que a coluna `year` indique o ano a que o registro se refere. Conferindo:

In [2]:
dados["year"].value_counts()

year
1973    64
2008    64
1999    64
2000    64
2001    64
2002    64
2003    64
2004    64
2005    64
2006    64
2007    64
2009    64
1997    64
2010    64
2011    64
2012    64
2013    64
2014    64
2015    64
2016    64
2017    64
2018    64
1998    64
1996    64
1974    64
1984    64
1975    64
1976    64
1977    64
1978    64
1979    64
1980    64
1981    64
1982    64
1983    64
1985    64
1995    64
1986    64
1987    64
1988    64
1989    64
1990    64
1991    64
1992    64
1993    64
1994    64
2019    64
Name: count, dtype: int64

Todos os anos aparecem 64 vezes, o que indica que a quantidade de registros por ano sempre é padronizada. Vale a pena converter a coluna para um tipo temporal, já que isso pode ser útil na plotagem de gráficos.

In [3]:
dados = dados.astype({"year": str})
dados["year"] = pd.to_datetime(dados["year"])

Indo para a variável `format`:

In [4]:
dados["format"].value_counts()

format
CD                                    141
DVD Audio                             141
Ringtones & Ringbacks                 141
Download Music Video                  141
Kiosk                                 141
CD Single                             141
Download Single                       141
SACD                                  141
Download Album                        141
Music Video (Physical)                141
Other Tapes                           141
8 - Track                             141
Vinyl Single                          141
LP/EP                                 141
Cassette Single                       141
Cassette                              141
Paid Subscriptions                     94
Limited Tier Paid Subscription         94
On-Demand Streaming (Ad-Supported)     94
Other Ad-Supported Streaming           94
Other Digital                          94
Paid Subscription                      94
SoundExchange Distributions            94
Synchronization            

Essa coluna, pelo nome das observações, é referente aos formatos em que as músicas podem ser distribuidas. A maioria aparece 141 vezes, e o restante 94 vezes, o que indica que há um número fixo de aparições de cada formato. Agora, analisando a coluna `metric`:

In [5]:
dados["metric"].value_counts()

metric
Value               1081
Value (Adjusted)    1081
Units                846
Name: count, dtype: int64

Essa coluna tem apenas três valores, que representa como foram contadas as vendas de alguma entrada na tabela: Por unidades vendidas, por valor da venda ou por valor ajustado para refletir a inflação atual. Vale notar que a métrica de unidades está em menor quantidade. Isso pode se relacionar aos formatos que também aparecem em menor quantidade na coluna de formatos. Para tirar a prova, qualquer um dos formatos pode ser usado:

In [6]:
dados[dados["format"] == "CD Single"]["metric"].value_counts()

metric
Units               47
Value               47
Value (Adjusted)    47
Name: count, dtype: int64

In [7]:
dados[dados["format"] == "Paid Subscription"]["metric"].value_counts()

metric
Value               47
Value (Adjusted)    47
Name: count, dtype: int64

Realmente, as contagens por unidade não existem para alguns formatos da tabela. Logo de cara com isso já é possível perceber que usar as métricas de unidades vendidas não seria adequado para fazer uma análise que inclua todas as mídias da base de dados.
Partindo agora para a análise da coluna `number_of_records`:

In [8]:
dados["number_of_records"].value_counts()

number_of_records
1    3008
Name: count, dtype: int64

Todos os valores da coluna são 1. Logo, ela não terá qualquer utilidade para a análise.

In [9]:
dados = dados.drop(columns = ["number_of_records"])

Por último, analisando a coluna `value_actual`:

In [10]:
dados["value_actual"].value_counts().sort_values(ascending = False)

value_actual
0.000000      77
1.300000       7
0.100000       7
1.500000       6
1.700000       6
              ..
647.930274     1
835.213587     1
865.719793     1
941.535965     1
5.386197       1
Name: count, Length: 1138, dtype: int64

Essa coluna é a que representa ou a quantidade de unidades vendidas ou o valor em dólares vendido, a depender da coluna `metric`. Quando a tabela é representada, é possível observar que alguns campos tem valores `NaN`:

In [11]:
dados["value_actual"]

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
          ...   
3003    6.205390
3004    5.198931
3005    6.339678
3006    5.386197
3007    6.795946
Name: value_actual, Length: 3008, dtype: float64

Uma possível hipótese para isso é que não hajam dados para as vendas de um determinado formato em um determinado ano, já que muitos deles não existiam em todo o período de tempo abrangido pela base. O melhor é observar a base melhor para ver se essa conclusão é plausível:

In [12]:
dados[dados["value_actual"].isna()].head(25)

,format,metric,year,value_actual
0,CD,Units,1973-01-01,NaN
1,CD,Units,1974-01-01,NaN
2,CD,Units,1975-01-01,NaN
3,CD,Units,1976-01-01,NaN
4,CD,Units,1977-01-01,NaN
5,CD,Units,1978-01-01,NaN
6,CD,Units,1979-01-01,NaN
7,CD,Units,1980-01-01,NaN
8,CD,Units,1981-01-01,NaN
9,CD,Units,1982-01-01,NaN


In [13]:
dados[dados["value_actual"].isna()].tail(25)

,format,metric,year,value_actual
2925,Synchronization,Value (Adjusted),1984-01-01,NaN
2926,Synchronization,Value (Adjusted),1985-01-01,NaN
2927,Synchronization,Value (Adjusted),1986-01-01,NaN
2928,Synchronization,Value (Adjusted),1987-01-01,NaN
2929,Synchronization,Value (Adjusted),1988-01-01,NaN
2930,Synchronization,Value (Adjusted),1989-01-01,NaN
2931,Synchronization,Value (Adjusted),1990-01-01,NaN
2932,Synchronization,Value (Adjusted),1991-01-01,NaN
2933,Synchronization,Value (Adjusted),1992-01-01,NaN
2934,Synchronization,Value (Adjusted),1993-01-01,NaN


In [14]:
dados[(dados["value_actual"].isna()) & (dados["format"] == "On-Demand Streaming (Ad-Supported)")]

,format,metric,year,value_actual
1457,On-Demand Streaming (Ad-Supported),Value,1973-01-01,NaN
1458,On-Demand Streaming (Ad-Supported),Value,1974-01-01,NaN
1459,On-Demand Streaming (Ad-Supported),Value,1975-01-01,NaN
1460,On-Demand Streaming (Ad-Supported),Value,1976-01-01,NaN
1461,On-Demand Streaming (Ad-Supported),Value,1977-01-01,NaN
...,...,...,...,...
2571,On-Demand Streaming (Ad-Supported),Value (Adjusted),2006-01-01,NaN
2572,On-Demand Streaming (Ad-Supported),Value (Adjusted),2007-01-01,NaN
2573,On-Demand Streaming (Ad-Supported),Value (Adjusted),2008-01-01,NaN
2574,On-Demand Streaming (Ad-Supported),Value (Adjusted),2009-01-01,NaN


In [15]:
dados[(dados["value_actual"].isna()) & (dados["format"] == "Download Album")]

,format,metric,year,value_actual
564,Download Album,Units,1973-01-01,NaN
565,Download Album,Units,1974-01-01,NaN
566,Download Album,Units,1975-01-01,NaN
567,Download Album,Units,1976-01-01,NaN
568,Download Album,Units,1977-01-01,NaN
...,...,...,...,...
2235,Download Album,Value (Adjusted),1999-01-01,NaN
2236,Download Album,Value (Adjusted),2000-01-01,NaN
2237,Download Album,Value (Adjusted),2001-01-01,NaN
2238,Download Album,Value (Adjusted),2002-01-01,NaN


Como é possível ver, os formatos usados para teste tem `NaN` em seus valores em anos nos quais eles não existiam. Logo, é plausível converter eles para 0.

In [16]:
dados["value_actual"] = dados["value_actual"].fillna(0)
dados["value_actual"].sort_values(ascending = True)

2143       -7.650944
1062       -5.300000
2144       -2.273770
1063       -1.600000
169        -1.500000
            ...     
2003    17115.944823
1999    17905.400687
2002    18635.677447
2001    19618.928145
2000    19667.327786
Name: value_actual, Length: 3008, dtype: float64

Resolvido o problema dos Na, aparece mais um problema com a coluna: Ela tem números negativos. Inspecionando para quais coluhas eles existem:

In [17]:
dados[dados["value_actual"] < 0]

,format,metric,year,value_actual
63,CD Single,Units,1989-01-01,-0.100000
169,Cassette Single,Units,2001-01-01,-1.500000
170,Cassette Single,Units,2002-01-01,-0.500000
463,DVD Audio,Units,2013-01-01,-0.054735
909,CD Single,Value,1989-01-01,-0.700000
1062,Cassette Single,Value,2001-01-01,-5.300000
1063,Cassette Single,Value,2002-01-01,-1.600000
1121,DVD Audio,Value,2013-01-01,-0.495804
2037,CD Single,Value (Adjusted),1989-01-01,-1.443225
2143,Cassette Single,Value (Adjusted),2001-01-01,-7.650944


São poucas as colunas que os tem, e eles são muito próximos do 0, enquanto normalmente os valores dessa coluna são bem maiores do que 0:

In [18]:
dados[dados["metric"] == "Value (Adjusted)"]["value_actual"].mean()

586.2608874992877

In [19]:
dados[dados["metric"] == "Value (Adjusted)"]["value_actual"].std()

2171.6024345669603

In [20]:
dados[dados["metric"] == "Value"]["value_actual"].mean()

346.42174998355966

In [21]:
dados[dados["metric"] == "Value"]["value_actual"].std()

1366.8893956791917

Tendo em vista isso, é plausível assumir que eles são erros e que não há problema em eliminá-los:

In [22]:
dados.loc[dados["value_actual"] < 0, "value_actual"] = 0

In [23]:
dados["value_actual"].value_counts()

value_actual
0.000000      1746
0.100000         7
1.300000         7
1.500000         6
1.700000         6
              ... 
527.000000       1
313.000000       1
14.300000        1
57.300000        1
5.386197         1
Name: count, Length: 1126, dtype: int64

Agora, a coluna `value_actual` está corrigida. Para analisar de forma bem superficial como são os valores para cada tipo de formato, é possível usar um gráfico facetado:

In [24]:
alt.Chart(dados).mark_line().encode(
    x = "year",
    y = "value_actual",
    color = "metric",
    facet = alt.Facet("format", columns = 5)
).properties(width = 160, height = 160)

alt.Chart(...)

É possível observar que alguns formatos só passam a ter dados depois de certos períodos de tempo (ou até certos períodos de tempo), o que é completamente plausível dentro do contexto dos dados. Um outro fator importante é ver se de alguma forma os valores ajustados para inflação conflitam proporcionalmente com os valores originais. Um simples gráfico de barras empilhadas pode responder esse questionamento:

In [25]:
alt.Chart(dados).mark_bar().encode(
    x = "year(year):N",
    y = alt.Y("value_actual").stack("normalize"),
    color = "format",
    row = "metric"
)

alt.Chart(...)

As proporções entre os valores permanecem as mesmas, logo não é prejudicial usar os valores ajustados par inflação para analisar as mudanças pelo tempo (pelo contrário é até preferível, já que dará uma noção melhor sobre como um período se sai melhor ou pior que outro sem ter o ruído da diferença do valor do dinheiro). No gráfico facetado, ficou visível que existe um formato chamado `Paid Subscriptions` e outro formato chamado `Paid Subscription`, com uma diferença de plural apenas. É pertinente analisar se eles se referem a mesma coisa ou coisas diferentes:

In [26]:
dados[dados["format"] == "Paid Subscriptions"].value_counts()

format              metric  year        value_actual
Paid Subscriptions  Units   1973-01-01  0.000000        2
                            1993-01-01  0.000000        2
                            1995-01-01  0.000000        2
                            1996-01-01  0.000000        2
                            1997-01-01  0.000000        2
                            1998-01-01  0.000000        2
                            1999-01-01  0.000000        2
                            1974-01-01  0.000000        2
                            2001-01-01  0.000000        2
                            2002-01-01  0.000000        2
                            2003-01-01  0.000000        2
                            2004-01-01  0.000000        2
                            2005-01-01  1.300000        2
                            2006-01-01  1.700000        2
                            2007-01-01  1.800000        2
                            2008-01-01  1.600000        2
                   

In [27]:
dados[dados["format"] == "Paid Subscription"].value_counts()

format             metric            year        value_actual
Paid Subscription  Value             1973-01-01  0.000000        1
                   Value (Adjusted)  1985-01-01  0.000000        1
                                     1994-01-01  0.000000        1
                                     1993-01-01  0.000000        1
                                     1992-01-01  0.000000        1
                                                                ..
                   Value             2002-01-01  0.000000        1
                                     2001-01-01  0.000000        1
                                     2000-01-01  0.000000        1
                                     1999-01-01  0.000000        1
                   Value (Adjusted)  2019-01-01  5934.397625     1
Name: count, Length: 94, dtype: int64

A difereça que se mostrou foi que um tem apenas métricas em unidades e o outro tem métricas em valores. Logo, é aceitável renomear um deles para o outro, tendo assim apenas um formato. 

In [28]:
dados.loc[dados["format"] == "Paid Subscription", "format"] = "Paid Subscriptions"

Após essa análise, existe a certeza de que a base será boa para ilustrar o que foi desejado na parte A, tendo em vista que alguns ajustes foram feitos. Já pelo gráfico facetado é possível ter uma pequena noção de como será o gráico e como ele será efetivo em mostrar o uso dos formatos de distribuição de áudio ao longo do tempo.

# Parte C

Com a ideia do que será representado em mente e com a base de dados limpa e sem ambiguidades, é preciso planejar como será executada a ideia.

O público alvo da peça gráfica será o público geral. Tomei essa decisão porque o entendimento geral do assunto não requer nenhum grande conhecimento prévio, logo o mais coerente é deixar a peça acessível para todos, e prezar pela facilidade de entender o que será mostrado (no caso, fazer de uma forma que seja compreensível como a indústria musical mudou). Não faz sentido, pelo menos no contexto de deixar acessível, encher a peça com muitas informações que são inúteis para quem quiser ver apenas o panorama total. 

Ao mesmo tempo, não é interessante esconder algumas informações que possam interessar à alguém curioso. Por isso, a peça gráfica que farei terá dois grandes gráficos: o principal, que mostrará de forma mais simples como mudou a indústria, e um mais complexo, que vai permitir que o usuário veja valores e filtre informações que ele quiser. O principal será exibido logo no começo da peça, chamando mais a atenção, enquanto o segundo será mais voltado para quem se interessar, assumindo papel secundário no contexto da peça. 

Como desejo usar alguns elementos de interatividade no segundo gráfico (e apenas um dispensável no primeiro), a peça gráfica será uma página visualizável em um navegador de internet. Como o tópico abordado não é sensível, e o objetivo é mostrar a trajetória do mercado musical, a peça terá expressividade neutra, apenas mostrando o que aconteceu.

O gráfico mais simples será um gráfico de área, com o eixo x representando a passagem de tempo e o y representando as vendas de cada formato. Ele terá ícones represetando as mídias e anotações representando pontos importantes para o contexto. As cores das áreas irão representar o formato de áudio, e isso é o que mostrará as mudanças do mercado. Como a base de dados tem 23 formatos, é importante agrupar eles em grupos que abranjam formatos parecidos, para evitar a criação de uma peça lotada de informações:

In [29]:
conversão = {'CD': "CD",
 'DVD Audio': "CD",
 'Ringtones & Ringbacks': "Download",
 'Download Music Video': "Download",
 'Kiosk': "Download",
 'CD Single': "CD",
 'Download Single': "Download",
 'SACD': "CD",
 'Download Album': "Download",
 'Music Video (Physical)': "Clipe",
 'Other Tapes': "Cassete",
 '8 - Track': "Cassete",
 'Vinyl Single': "Vinil",
 'LP/EP': "Vinil",
 'Cassette Single': "Cassete",
 'Cassette': "Cassete",
 'Paid Subscriptions': "Streaming",
 'Limited Tier Paid Subscription': "Streaming",
 'On-Demand Streaming (Ad-Supported)': "Streaming",
 'Other Ad-Supported Streaming': "Streaming",
 'Other Digital': "Download",
 'SoundExchange Distributions': "Download",
 'Synchronization': "Download"}
dados["midia"] = dados.apply(lambda l: conversão[l["format"]], axis = 1)
dados["midia"].value_counts()

midia
Download     987
CD           564
Cassete      564
Streaming    470
Vinil        282
Clipe        141
Name: count, dtype: int64

As cores de cada área foram selecionadas de um modo que, quando possivel, fossem relacionadas à midia que representam:

In [30]:
ordem_midia = ["Vinil", "Cassete", "CD", "Clipe", "Download", "Streaming"]
cores_midia = ["#404040", "#f0d23e", "#C7C7C7", "#F04032", "#2E26F0", "#32F03F"]

O gráfico secundário será de barras empilhadas, o que adiciona uma distinção dos anos mais clara. Ele exibirá os números de cada formato em cada ano, e será possível filtrar quais formatos aparecem a qualquer momento. Cada formato terá sua própria cor, respeitando as cores de cada grupo. Elas não tem tanta importância, já que o gráfico terá uma opção de filtrar os formatos, então algumas cores não serão facilmente distinguiveis.

In [31]:
ordem_formato = [
    "Vinyl Single",
    "LP/EP",
    "Other Tapes",
    "8 - Track",
    "Cassette Single",
    "Cassette",
    "CD",
    "DVD Audio",
    "CD Single",
    "SACD",
    "Music Video (Physical)",
    "Ringtones & Ringbacks",
    "Download Music Video",
    "Kiosk",
    "Download Single",
    "Download Album",
    "Other Digital",
    "SoundExchange Distributions",
    "Synchronization",
    "Paid Subscriptions",
    "On-Demand Streaming (Ad-Supported)",
    "Other Ad-Supported Streaming",
    "Limited Tier Paid Subscription"
]
cores_formato = [
    "#222f2d",
    "#404040",
    "#70683f",
    "#FFFE00",
    "#B3B32E",
    "#f0d23e",
    "#C7C7C7",
    "#8D8C8F",
    "#C2C0C4",
    "#7F8080",
    "#FF0000",
    "#275AF5",
    "#1B18DE",
    "#5B1CFA",
    "#187EDE",
    "#187EDE",
    "#187EDE",
    "#1F48C2",
    "#7193F7",
    "#39F50B",
    "#02DE1D",
    "#7BDE02",
    "#03FA67"
]

Os dois gráficos serão feitos em uma mesma página, mas também poderão funcionar bem sozinhos.

## Parte D

### Gráfico 1
Primeiro, produzirei o gráfico principal. O eixo X terá os anos, o eixo Y terá os valores de vendas e as cores serão de acordo com a mídia:

In [32]:
eixo_x_1 = alt.X("year(year):T").title("Ano").axis(
    values = (1975, 1980, 1985, 1990, 1995, 2000, 2005, 2010, 2015)
)
eixo_y_1 = alt.Y("value_actual:Q").title("Vendas (Ajustado para inflação)").scale(domain = (0, 25000)).axis(
    tickCount = 5, 
    labelExpr = '"$" + datum.value + "M"'
)
cores_1 = alt.Color("midia:N").scale(domain = ordem_midia, range = cores_midia).title("Mídia")

base = alt.Chart(
    dados,
    title = alt.Title(
        text = "Vendas da indústria musical americana.",
        subtitle = "A história da predominância dos formatos de distribução musical mostrada nos números de vendas de cada formato, de 1973 à 2019."
    )
).transform_filter(
    "datum.metric == 'Value (Adjusted)'"
).mark_area().encode(
    eixo_x_1,
    eixo_y_1,
    cores_1,
    alt.Detail("format")
).properties(
    width = 1080, 
    height = 600
)
base

alt.Chart(...)

Para colocar os ícones representando as mídias, é preciso usar a função `mark_image` com um dataframe contendo as posições das imagens, seus links, e no caso do gráfico um texto para ser exibido quando o usuário passar o mouse em cima da imagem:

In [33]:
imagens_1 = pd.DataFrame.from_records(
    [{
        "year": "1978",
        "value_actual": 4000,
        "img": "https://cdn-icons-png.flaticon.com/512/1378/1378161.png",
        "Sobre": """A tecnologia por trás dos vinis existe desde 1880, mas na década de 40 eles adquiriram a cara 
                   conhecida hoje em dia. Eles foram responsáveis por estabelecer o modelo de álbuns e singles 
                   usado até hoje, devido a suas limitações de tamanho."""
    },
    {
        "year": "1989",
        "value_actual": 4000,
        "img": "https://cdn-icons-png.flaticon.com/512/1969/1969405.png",
        "Sobre": """As fitas cassete são do começo dos anos 60, e tem tamanhos menores que os grandes discos de 
                   vinil, o que permitiu a criação de reprodutores portáteis, alavancando o formato. Eles também
                   eram graváveis em casa, sem a necessidade de aparelhos profissionais.
                    """
    },
    {
        "year": "1999",
        "value_actual": 11000,
        "img": "https://cdn-icons-png.flaticon.com/512/4910/4910593.png",
        "Sobre": """Os CDs surgiram já com o propósito de substituir os vinis, apresentando tamanho reduzido e com
                   isso, portabilidade. A qualidade de som desse formato era superior a dos cassetes, outro fator
                   importante na popularização da mídia."""
    }]
)
imagens_2 = pd.DataFrame.from_records([
    {
        "year": "2012",
        "value_actual": 2750,
        "img": "https://cdn-icons-png.flaticon.com/512/1383/1383261.png",
        "Sobre": """A venda de músicas na internet começou simulando o modelo físico: O consumidor pagava
                   individualmente por álbum ou faixa, tendo acesso ilimitado à música comprada. Aparelhos como os
                   MP3 players permitiam que a música fosse ouvida de qualquer lugar.
                   """
    },
    {
        "year": "2017-07",
        "value_actual": 3000,
        "img": "https://cdn-icons-png.flaticon.com/512/59/59860.png",
        "Sobre": """O formato de streaming é construido em um modelo que permite que o ouvinte ouça qualquer música 
                   desejada dentro de um catálogo, mediante a exibição de anúncios ou do pagamento de planos 
                   de assinatura. Rádios digitais também se encaixam na categoria.
                   """
    },
    {
        "year": "2012-02",
        "value_actual": 23000,
        "img": "https://cdn-icons-png.flaticon.com/512/5968/5968946.png",
        "Sobre": "Eu sei que você alguma vez já baixou uma música pirata..."
    }]
)
graf_imagens_1 = alt.Chart(imagens_1).mark_image(
    width = 85,
    height = 85
).encode(
    x = "year(year):T", 
    y = "value_actual", 
    url = "img",
    tooltip = ("Sobre",)
)
#Esse segundo gráfico é feito porque essas imagens tem tamanho diferente das outras.
graf_imagens_2 = alt.Chart(imagens_2).mark_image(
    width = 70,
    height = 70
).encode(
    x = "yearmonth(year):T", 
    y = "value_actual", 
    url = "img",
    tooltip = ("Sobre",)
)
imagens = graf_imagens_1 + graf_imagens_2
base + imagens

alt.LayerChart(...)

Agora, falta adicionar uma anotação explicando a queda das vendas. Para isso será usada uma linha de marcação e texto. A linha é adicionada com a função `mark_rule`, que recebe um dataframe simples com a posição a se marcar a linha:

In [34]:
regua = alt.Chart(
    pd.DataFrame.from_records([{"year": "1999-11"}])
).mark_rule(
    color = "darkred", 
    strokeDash = (12, 4), 
    strokeWidth = 2
).encode(
    x = "yearmonth(year):T"
)
base + regua

alt.LayerChart(...)

A anotação de texto segue o mesmo esquema das outras: recebe um dataframe com a posição e o texto. Como o texto é extenso e não ficaria bom em uma linha só, é preciso usar o método `transform_calculate` para realizar uma simples operação de quebra de linha.

In [35]:
textos = pd.DataFrame({
    "texto": ["No final de 1999, o software Napster \n foi criado. Ele permitia o compartilhamento\n"\
              "de arquivos por uma rede P2P, e acabou\nfocado no compartilhamento de MP3s.\n"\
              "Seu efeito se reflete na queda de vendas\nque a industria sofreu na década subsequente."],
    "year": "2000-04",
    "value_actual": 24650
})
anotacao = alt.Chart(
    textos
).mark_text(
    align = "left",
    color = "darkred"
).transform_calculate(
    texto = "split(datum.texto, '\\n')"
).encode(
    x = "yearmonth(year):T", 
    y = "value_actual", 
    text = "texto"
)
base + anotacao

alt.LayerChart(...)

Agora, basta juntar todos os layers do gráfico e configurar fontes e outros detalhes:

In [36]:
final = (base + imagens + regua + anotacao).configure(
    font = "helvetica",
    background = "transparent"
).configure_legend(
    strokeColor = "black",
    orient = "top-left",
    padding = 8,
    cornerRadius = 3,
    titleAlign = "center",
    titleAnchor = "middle",
    titleFontSize = 13.5,
    fillColor = "#FFFFFF",
    labelFontSize = 11.5 
).configure_axis(
    gridColor = "#000000",
    gridOpacity = 0.2,
    domainColor = "#000000",
    labelFontSize = 11.5,
    titleFontSize = 13.5
).configure_title(
    anchor = "start",
    fontSize = 25,
    subtitleFontSize = 18
).configure_view(
    stroke = "#000000"
)
final

alt.LayerChart(...)

Essa versão do gráfico tem título e subtítulo, adicionados para ela ter sentido sozinha. Na versão que será colocada na página, esses elementos serão inexistentes. Para salvar como html, é só usar a função `save`.

In [37]:
final.save("grafico_1.html")

### Gráfico 2

No gráfico 2, os canais mapeados serão os mesmos. A diferença está que ele terá um `tooltip`, assim como as imagens. Para que isso seja possível, é preciso usar a função `transform_stack`, feita para gerar gráficos com barras empilhadas e tooltips. Gerando o gráfico sem filtros interativos:

In [38]:
eixo_x_2 = alt.X("year(year):N").title("Ano").axis(values = (1975, 1980, 1985, 1990, 1995, 2000, 2005, 2010, 2015))
eixo_y_2 = alt.Y("valor1:Q").title("Vendas (Ajustado para inflação)").scale(domain = (0, 25000)).axis(
    tickCount = 10, 
    labelExpr = '"$" + datum.value + "M"'
)
cores_2 = alt.Color("format:N").scale(domain = ordem_formato, range = cores_formato).title("Formato")
tooltip_2 = [
    alt.Tooltip('Formato:N', title = None), 
    alt.Tooltip("Mídia:N", title = None), 
    alt.Tooltip("Vendas:N", title = None)
]

final_2 = alt.Chart(
    dados,
    title = alt.Title(
        text = "Vendas da indústria musical americana.",
        subtitle = ["A história da predominância dos formatos de distribução musical ", 
                    "mostrada nos números de vendas de cada formato, de 1973 à 2019.",
                    "(Valores são ajustados de acordo com a inflação)"]
    )
).transform_stack( #A função irá "quebrar" a variável em um início e um fim, indicando onde a barra é posicionada.
    stack = "value_actual", 
    as_ = ("valor1", "valor2"),
    groupby = ("year", "metric"),
    sort = (alt.SortField("midia", "descending"),)
).transform_filter(
    'datum.metric == "Value (Adjusted)"'
).transform_calculate( #Produz os textos mostrados no Tooltip
    Formato = 'datum.format',
    Mídia = 'datum.midia',
    Vendas = 'round(datum.valor2 - datum.valor1) + " milhões de dólares no ano de " + year(datum.year)'
).mark_bar(
).encode(
    eixo_x_2,
    eixo_y_2,
    cores_2,
    tooltip_2,
    y2 = "valor2:Q" 
).properties(
    width = 600, 
    height = 720
).configure(
    font = "helvetica",
    background = "transparent"
).configure_legend(
    strokeColor = "black",
    orient = "right",
    padding = 8,
    cornerRadius = 3,
    titleAlign = "center",
    titleAnchor = "middle",
    titleFontSize = 13.5,
    fillColor = "#FFFFFF",
    labelFontSize = 11.5 
).configure_axis(
    gridColor = "#000000",
    gridOpacity = 0.2,
    domainColor = "#000000",
    labelFontSize = 11.5,
    titleFontSize = 13.5
).configure_title(
    anchor = "start",
    fontSize = 25,
    subtitleFontSize = 18
).configure_view(
    stroke = "#000000"
)
final_2

alt.Chart(...)

Esse gráfico tera duas formas de interação: um seletor que permitirá ver os valores ajustados para inflação ou os originais, e uma legenda que servirá como botão para selecionar quais formatos exibir. Primeiramente, a base de dados será alterada para que o seletor esteja em português:

In [39]:
traducao = {
    "Value": "Valor original",
    "Value (Adjusted)": "Valor ajustado"
}
dados_2 = dados[(dados["metric"] == "Value") | (dados["metric"] == "Value (Adjusted)")]
dados_2.loc[:, "metric"] = dados_2.apply(lambda l: traducao[l["metric"]], axis = 1)
dados_2

,format,metric,year,value_actual,midia
846,CD,Valor original,1973-01-01,0.000000,CD
847,CD,Valor original,1974-01-01,0.000000,CD
848,CD,Valor original,1975-01-01,0.000000,CD
849,CD,Valor original,1976-01-01,0.000000,CD
850,CD,Valor original,1977-01-01,0.000000,CD
...,...,...,...,...,...
3003,Vinyl Single,Valor ajustado,2015-01-01,6.205390,Vinil
3004,Vinyl Single,Valor ajustado,2016-01-01,5.198931,Vinil
3005,Vinyl Single,Valor ajustado,2017-01-01,6.339678,Vinil
3006,Vinyl Single,Valor ajustado,2018-01-01,5.386197,Vinil


Para fazer o seletor, é preciso de um objeto `binding_select`. Ele terá listadas as duas opções possíveis de valores. Para dizer ao gráfico que existe um filtro é preciso usar o objeto `selection_point`, que será atrelado à coluna `metric` e ao seletor criado:

In [40]:
dropdown_valores = alt.binding_select(
    options = ("Valor ajustado", "Valor original"),
    name = "Valores das vendas: "
)
selecao_valores = alt.selection_point(
    fields = ("metric",),
    bind = dropdown_valores
)

Fazer a legenda é mais simples: é preciso apenas de um objeto `selection_point`, que será atrelado à coluna `format` e a legenda, diretamente:

In [41]:
selecao_formatos = alt.selection_point(
    fields = ("format",),
    bind = "legend"
)

No gráfico, a única diferença será no método `transform_filter`, que receberá os objetos criados acima, e no método `add_param`, que vai adicionar os seletores ao gráfico:

In [42]:
eixo_y_2 = alt.Y("valor1:Q").title("Vendas (Ajustado para inflação)").axis(
    tickCount = 10, 
    labelExpr = '"$" + datum.value + "M"'
)

final_2 = alt.Chart(
    dados_2,
    title = alt.Title(
        text = "Vendas da indústria musical americana.",
        subtitle = ["A história da predominância dos formatos de distribução musical ", 
                    "mostrada nos números de vendas de cada formato, de 1973 à 2019.",
                    "(Valores são ajustados de acordo com a inflação)"]
    )
).transform_stack( #A função irá "quebrar" a variável em um início e um fim, indicando onde a barra é posicionada.
    stack = "value_actual", 
    as_ = ("valor1", "valor2"),
    groupby = ("year", "metric"),
    sort = (alt.SortField("midia", "descending"),)
).transform_filter(
    selecao_valores
).transform_filter(
    selecao_formatos
).transform_calculate( #Produz os textos mostrados no Tooltip
    Formato = 'datum.format',
    Mídia = 'datum.midia',
    Vendas = 'round(datum.valor2 - datum.valor1) + " milhões de dólares no ano de " + year(datum.year)'
).mark_bar(
).encode(
    eixo_x_2,
    eixo_y_2,
    cores_2,
    tooltip_2,
    y2 = "valor2:Q" 
).add_params(
    selecao_valores,
    selecao_formatos
).properties(
    width = 600, 
    height = 720
).configure(
    font = "helvetica",
    background = "transparent"
).configure_legend(
    strokeColor = "black",
    orient = "right",
    padding = 8,
    cornerRadius = 3,
    titleAlign = "center",
    titleAnchor = "middle",
    titleFontSize = 13.5,
    fillColor = "#FFFFFF",
    labelFontSize = 11.5 
).configure_axis(
    gridColor = "#000000",
    gridOpacity = 0.2,
    domainColor = "#000000",
    labelFontSize = 11.5,
    titleFontSize = 13.5
).configure_title(
    anchor = "start",
    fontSize = 25,
    subtitleFontSize = 18
).configure_view(
    stroke = "#000000"
)
final_2

alt.Chart(...)

Por fim, para salvar:

In [43]:
final_2.save("grafico_2.html")

Com os gráficos em mãos, é preciso fazer a página HTML. O HTML básico que é gerado no método `save` vem apenas com o que é necessário para exibir o gráfico, o que inclui um pouco de CSS e um código em JavaScript, que atrela o gráfico a uma `div`. Logo, copiando esses trechos específicos, é possível exibir o gráfico em uma outra página HTML, posicionando o elemento `div` onde for mais pertinente, sem grandes dificuldades. Mais especificamente, o que é responável por gerar o gráfico são esses trechos:
```css
<style>
    #vis.vega-embed {
      width: 100%;
      display: flex;
    }

    #vis.vega-embed details,
    #vis.vega-embed details summary {
      position: relative;
    }
</style>
```
```html
<script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
<script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@5.8.0"></script>
<script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
  
<div id="vis"></div>
<script>
  (function(vegaEmbed) {
    var spec = {"aqui há uma linha com toda a especificação do gráfico"};
    var embedOpt = {"mode": "vega-lite"};
    function showError(el, error){
        el.innerHTML = ('<div style="color:red;">'
                        + '<p>JavaScript Error: ' + error.message + '</p>'
                        + "<p>This usually means there's a typo in your chart specification. "
                        + "See the javascript console for the full traceback.</p>"
                        + '</div>');
        throw error;
    }
    const el = document.getElementById('vis');
    vegaEmbed("#vis", spec, embedOpt)
      .catch(error => showError(el, error));
  })(vegaEmbed);
</script>
```

Com os gráficos em mãos, a etapa restante é compor o HTML final da peça gráfica. A composição dele é a seguinte:

    -Titulo
    -Gráfico principal
    -Textos explicando os formatos
    -Gráfico secundário
    
Os gráficos usados nela tiveram o seu título removido, já que seria redundante dois gráficos com o mesmo título. A função de explicar o que é o gráfico ficou para a própria página, no lugar. Os textos organizados em blocos foram feitos usando uam tag `div`, com a propriedade CSS de agrupar mais `divs` especiais. O resultado, que é a peça gráfica final, pode ser observado no arquivo `pecagrafica.html`.